In [30]:
from math import ceil, log10
from random import seed, randint, random
from numpy import array, cumsum, array_equal
from keras.models import Sequential
from keras.layers import LSTM, RepeatVector, TimeDistributed, Bidirectional
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
from numpy import argmax

In [14]:
# create a sequence of random numbers in [0,1]
X = array([random() for _ in range(10)])

In [15]:
# calculate cut-off value to change class values
limit = 10/4.0

In [16]:
y = array([0 if x < limit else 1 for x in cumsum(X)])

In [17]:
# create a cumulative sum sequence
def get_sequence(n_timesteps):
    # create a sequence of random numbers in [0,1]
    X = array([random() for _ in range(n_timesteps)])
    # calculate cut-off value to change class values
    limit = n_timesteps/4.0
    #determine the class outcome for each item in cumulative sequence
    y = array([0 if x < limit else 1 for x in cumsum(X)])
    return X, y

In [20]:
X, y = get_sequence(10)
print(X)
print(y)

[0.5598399  0.5747337  0.55345078 0.74665299 0.53604792 0.64358086
 0.99948499 0.72737896 0.39973774 0.25047173]
[0 0 0 0 1 1 1 1 1 1]


In [21]:
# create multiple samples of cumulative sum sequences
def get_sequences(n_sequences, n_timesteps):
    seqX, seqY = list(), list()
    #create and store sequences
    for _ in range(n_sequences):
        X, y = get_sequence(n_timesteps)
        seqX.append(X)
        seqY.append(y)
    #reshape input and output for lstm
    seqX = array(seqX).reshape(n_sequences, n_timesteps, 1)
    seqY = array(seqY).reshape(n_sequences, n_timesteps, 1)
    return seqX, seqY

In [22]:
# define problem
n_timesteps = 10

In [24]:
# define LSTM
model = Sequential()
model.add(Bidirectional(LSTM(50, return_sequences=True), input_shape=(n_timesteps, 1)))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 10, 100)           20800     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 1)             101       
Total params: 20,901
Trainable params: 20,901
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
# train LSTM
X, y = get_sequences(50000, n_timesteps)

In [26]:
model.fit(X, y, epochs=1, batch_size=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
50000/50000 [==============================] - 68s 1ms/step - loss: 0.0513 - acc: 0.9812


In [27]:
# evaluate LSTM
X, y = get_sequences(100, n_timesteps)
loss, acc = model.evaluate(X, y, verbose=0)
print('Loss: %f, Accuracy: %f' % (loss, acc*100))

Loss: 0.024940, Accuracy: 99.199999


In [31]:
# make predictions
for _ in range(10):
    X, y = get_sequences(1, n_timesteps)
    yhat = model.predict_classes(X, verbose=0)
    exp, pred = y.reshape(n_timesteps), yhat.reshape(n_timesteps)
    print( 'y=%s, yhat=%s, correct=%s' % (exp, pred, array_equal(exp,pred)))

y=[0 0 0 0 0 0 1 1 1 1], yhat=[0 0 0 0 0 0 1 1 1 1], correct=True
y=[0 0 0 0 1 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=True
y=[0 0 0 0 0 0 1 1 1 1], yhat=[0 0 0 0 0 0 1 1 1 1], correct=True
y=[0 0 0 1 1 1 1 1 1 1], yhat=[0 0 0 1 1 1 1 1 1 1], correct=True
y=[0 0 0 0 0 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=False
y=[0 0 0 0 0 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=False
y=[0 0 0 1 1 1 1 1 1 1], yhat=[0 0 0 1 1 1 1 1 1 1], correct=True
y=[0 0 0 1 1 1 1 1 1 1], yhat=[0 0 0 1 1 1 1 1 1 1], correct=True
y=[0 0 0 0 1 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=True
y=[0 0 0 1 1 1 1 1 1 1], yhat=[0 0 0 1 1 1 1 1 1 1], correct=True
